# Managed RAG with Vertex AI RAG Engine

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayoisio/genai-on-google-cloud/blob/main/chapter-2/colabs/05_vertex_ai_rag_engine.ipynb)

**Estimated Time**: 15-20 minutes

**Prerequisites**: Google Cloud project with billing enabled, Vertex AI API enabled

---

## Overview

Vertex AI RAG Engine provides a fully managed solution for Retrieval-Augmented Generation. This notebook demonstrates:

1. **Create a RAG corpus** for document storage and indexing
2. **Upload and index documents** from various sources
3. **Query with RAG retrieval** using GenerativeModel
4. **Compare DIY vs managed RAG** approaches

RAG Engine handles document chunking, embedding generation, vector indexing, and retrieval automatically.

## 1. Setup & Authentication

In [ ]:
# @title Install Dependencies
!pip install --upgrade google-cloud-aiplatform google-cloud-storage -q

In [ ]:
# @title Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()
print("✓ Authentication successful")

In [ ]:
# @title Configure Your Project
PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Validate project ID
if PROJECT_ID == "your-project-id":
    raise ValueError("Please set your PROJECT_ID above")

print(f"✓ Project: {PROJECT_ID}")
print(f"✓ Location: {LOCATION}")

In [ ]:
# @title Initialize Vertex AI
import vertexai
from vertexai.preview import rag  # RAG is in preview module
from vertexai.generative_models import GenerativeModel, Tool

vertexai.init(project=PROJECT_ID, location=LOCATION)

print(f"✓ Vertex AI initialized")
print(f"✓ RAG Engine ready")

## 2. Understanding RAG Engine Architecture

Vertex AI RAG Engine provides:

- **RAG Corpus**: A managed container for your documents and their embeddings
- **RAG Files**: Individual documents uploaded to a corpus
- **Automatic Processing**: Chunking, embedding, and indexing handled automatically
- **Retrieval Tool**: Integration with GenerativeModel for grounded responses

```mermaid
flowchart LR
    A[Documents] --> B[RAG Corpus]
    B --> C[Embeddings]
    C --> D[Vector Index]
    D --> E[Retrieval]
    E --> F[Gemini]
    F --> G[Response]
```

## 3. Create a RAG Corpus

A RAG Corpus is a container that stores your documents along with their embeddings for efficient retrieval.

In [ ]:
# @title Create a new RAG corpus
CORPUS_DISPLAY_NAME = "chapter2_demo_corpus"  # @param {type:"string"}
CORPUS_DESCRIPTION = "Demo corpus for Chapter 2 RAG examples"  # @param {type:"string"}

# Configure embedding model for the corpus
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-005"
)

# Create the corpus
rag_corpus = rag.create_corpus(
    display_name=CORPUS_DISPLAY_NAME,
    description=CORPUS_DESCRIPTION,
    embedding_model_config=embedding_model_config,
)

print(f"✓ Created RAG Corpus")
print(f"  Name: {rag_corpus.name}")
print(f"  Display Name: {rag_corpus.display_name}")

In [ ]:
# @title List existing corpora (optional)
corpora = rag.list_corpora()

print("📚 Existing RAG Corpora:")
for corpus in corpora:
    print(f"  - {corpus.display_name}: {corpus.name}")

## 4. Upload Documents to the Corpus

RAG Engine supports multiple document sources:
- **Cloud Storage**: GCS URIs for files
- **Google Drive**: Drive folder/file IDs
- **Inline text**: Direct text upload

Documents are automatically chunked, embedded, and indexed.

In [ ]:
# @title Create sample documents for upload
# We'll create sample documents inline for this demo

SAMPLE_DOCUMENTS = [
    {
        "display_name": "ml_fundamentals.txt",
        "content": """Machine Learning Fundamentals

Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed. The core idea is to develop algorithms that can access data and use it to learn for themselves.

There are three main types of machine learning:

1. Supervised Learning: Uses labeled data to train models. The algorithm learns from example input-output pairs to predict outputs for new inputs. Common applications include classification and regression.

2. Unsupervised Learning: Finds patterns in data without labels. The algorithm discovers hidden structures in unlabeled data. Common applications include clustering and dimensionality reduction.

3. Reinforcement Learning: Learns through rewards and penalties. An agent learns to make decisions by performing actions and receiving feedback. Common applications include game playing and robotics.
"""
    },
    {
        "display_name": "deep_learning_intro.txt",
        "content": """Introduction to Deep Learning

Deep learning uses neural networks with multiple layers (hence 'deep') to progressively extract higher-level features from raw input. For example, in image recognition, lower layers might identify edges, while higher layers identify concepts like faces or objects.

Training a neural network involves:
- Forward Propagation: Computing predictions by passing input through the network
- Loss Calculation: Measuring the error between predictions and actual values
- Backpropagation: Adjusting weights to minimize the loss

This process repeats over many iterations (epochs) until the model converges to an optimal solution. Deep learning has revolutionized fields like computer vision, natural language processing, and speech recognition.
"""
    },
    {
        "display_name": "rag_overview.txt",
        "content": """Retrieval-Augmented Generation (RAG)

RAG combines the power of large language models with external knowledge retrieval. Instead of relying solely on the model's trained knowledge, RAG retrieves relevant documents and uses them to generate more accurate, up-to-date responses.

Key Benefits of RAG:
- Reduced Hallucinations: Responses are grounded in real documents
- Source Citations: Can cite specific sources for information
- Easy Updates: Knowledge can be updated without retraining the model
- Domain Accuracy: Provides accurate information for specific domains

RAG is particularly valuable for enterprise applications where accuracy, currency, and traceability of information are critical. It bridges the gap between general LLM knowledge and specific organizational knowledge.
"""
    },
    {
        "display_name": "vertex_ai_features.txt",
        "content": """Vertex AI Platform Features

Vertex AI is Google Cloud's unified platform for machine learning and AI. It provides a comprehensive suite of tools for building, deploying, and managing AI applications.

Key Features:
- Generative AI: Access to Gemini models for text, code, and multimodal generation
- Model Garden: Pre-trained models for various tasks
- RAG Engine: Managed retrieval-augmented generation
- Vector Search: High-performance similarity search
- Feature Store: Centralized feature management
- Pipelines: MLOps workflow orchestration
- Experiments: Track and compare model versions

Vertex AI integrates seamlessly with BigQuery, Cloud Storage, and other Google Cloud services, enabling end-to-end AI workflows at enterprise scale.
"""
    }
]

print(f"✓ Prepared {len(SAMPLE_DOCUMENTS)} sample documents for upload")
for doc in SAMPLE_DOCUMENTS:
    print(f"  - {doc['display_name']}")

In [ ]:
# @title Save sample documents to Cloud Storage (for upload)
from google.cloud import storage
import uuid

# Create a unique bucket name
BUCKET_NAME = f"{PROJECT_ID}-rag-demo-{uuid.uuid4().hex[:8]}"

# Initialize storage client
storage_client = storage.Client(project=PROJECT_ID)

# Create bucket
try:
    bucket = storage_client.create_bucket(BUCKET_NAME, location=LOCATION)
    print(f"✓ Created bucket: {BUCKET_NAME}")
except Exception as e:
    bucket = storage_client.bucket(BUCKET_NAME)
    print(f"Using existing bucket: {BUCKET_NAME}")

# Upload documents
gcs_uris = []
for doc in SAMPLE_DOCUMENTS:
    blob = bucket.blob(f"documents/{doc['display_name']}")
    blob.upload_from_string(doc['content'], content_type='text/plain')
    gcs_uri = f"gs://{BUCKET_NAME}/documents/{doc['display_name']}"
    gcs_uris.append(gcs_uri)
    print(f"  Uploaded: {gcs_uri}")

print(f"\n✓ Uploaded {len(gcs_uris)} documents to Cloud Storage")

In [ ]:
# @title Import documents into RAG corpus
# Import documents from Cloud Storage into the corpus

# Configure chunking (optional - using defaults)
# RAG Engine will automatically chunk documents

response = rag.import_files(
    corpus_name=rag_corpus.name,
    paths=gcs_uris,
    chunk_size=512,  # Characters per chunk
    chunk_overlap=100,  # Overlap between chunks
)

print(f"✓ Import completed")
print(f"  Imported file count: {response.imported_rag_files_count}")

In [ ]:
# @title List files in the corpus
rag_files = rag.list_files(corpus_name=rag_corpus.name)

print("📄 Files in corpus:")
for file in rag_files:
    print(f"  - {file.display_name}")
    print(f"    Name: {file.name}")

## 5. Query the RAG Corpus

Now let's query our corpus using two methods:
1. **Direct Retrieval**: Get relevant chunks without generation
2. **RAG-Grounded Generation**: Use retrieved context for LLM response

In [ ]:
# @title Direct retrieval from corpus
QUERY = "What are the benefits of RAG?"  # @param {type:"string"}

# Retrieve relevant contexts
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
        )
    ],
    text=QUERY,
    similarity_top_k=3,
)

print(f"🔍 Query: '{QUERY}'\n")
print("Retrieved contexts:")
print("-" * 60)

for i, context in enumerate(response.contexts.contexts, 1):
    print(f"\n[{i}] Score: {context.distance:.4f}")
    print(f"Source: {context.source_uri}")
    print(f"Text: {context.text[:300]}...")

In [ ]:
# @title RAG-grounded generation with Gemini
# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(
                    rag_corpus=rag_corpus.name,
                )
            ],
            similarity_top_k=3,
        ),
    )
)

# Create model with RAG tool
rag_model = GenerativeModel(
    model_name="gemini-2.0-flash",
    tools=[rag_retrieval_tool],
)

print("✓ RAG-enabled model created")

In [ ]:
# @title Generate RAG-grounded response
QUESTION = "What is RAG and what are its key benefits for enterprise applications?"  # @param {type:"string"}

response = rag_model.generate_content(QUESTION)

print(f"🔍 Question: {QUESTION}\n")
print("=" * 60)
print("📝 RAG-Grounded Response:")
print("=" * 60)
print(response.text)

In [ ]:
# @title Test with multiple questions
test_questions = [
    "What are the three types of machine learning?",
    "How does deep learning training work?",
    "What features does Vertex AI provide?",
    "How does backpropagation work in neural networks?"
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"🔍 Question: {question}")
    print("-" * 60)
    
    response = rag_model.generate_content(question)
    print(f"📝 Answer: {response.text[:400]}...")

## 6. DIY RAG vs Managed RAG Comparison

Let's compare the approaches we've learned in this chapter.

In [ ]:
# @title Comparison: DIY RAG vs Vertex AI RAG Engine
comparison = """
┌─────────────────────────────────────────────────────────────────────────────┐
│                    DIY RAG vs Vertex AI RAG Engine                          │
├──────────────────────┬────────────────────────┬─────────────────────────────┤
│ Aspect               │ DIY RAG                │ Vertex AI RAG Engine        │
│                      │ (Notebook 04)          │ (This Notebook)             │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Setup Complexity     │ High - manage all      │ Low - managed service       │
│                      │ components yourself    │                             │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Chunking             │ Custom implementation  │ Automatic with config       │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Embeddings           │ Generate & store       │ Managed automatically       │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Vector Index         │ Build & maintain       │ Managed automatically       │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Retrieval            │ Custom similarity      │ Built-in retrieval_query    │
│                      │ search logic           │                             │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ LLM Integration      │ Manual prompt          │ Tool.from_retrieval()       │
│                      │ engineering            │                             │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Scaling              │ Manual infrastructure  │ Auto-scaling                │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Customization        │ Full control           │ Limited to config options   │
├──────────────────────┼────────────────────────┼─────────────────────────────┤
│ Best For             │ Custom requirements,   │ Quick deployment,           │
│                      │ specialized logic      │ production workloads        │
└──────────────────────┴────────────────────────┴─────────────────────────────┘
"""

print(comparison)

In [ ]:
# @title When to use each approach
guidance = """
📋 Choosing Your RAG Approach

✅ Use DIY RAG (Notebook 04) when:
   • You need custom chunking strategies
   • You require specialized embedding models
   • You need fine-grained control over retrieval logic
   • You're integrating with existing vector databases
   • You need custom re-ranking or filtering

✅ Use Vertex AI RAG Engine (This notebook) when:
   • You want rapid deployment to production
   • You prefer managed infrastructure
   • Standard chunking and embedding work for your use case
   • You need built-in scaling and reliability
   • You want tight integration with Gemini models

💡 Hybrid Approach:
   Many production systems use both - custom preprocessing
   with managed retrieval and generation.
"""

print(guidance)

## 7. Advanced Configuration (Reference)

RAG Engine provides various configuration options for fine-tuning performance.

In [ ]:
# @title Advanced RAG configuration patterns
ADVANCED_PATTERNS = '''
# 1. Custom Embedding Model
embedding_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-005",
    # Or use a custom endpoint:
    # endpoint="projects/{project}/locations/{location}/endpoints/{endpoint_id}"
)

# 2. Custom Chunking Configuration
# When importing files:
response = rag.import_files(
    corpus_name=corpus.name,
    paths=["gs://bucket/document.pdf"],
    chunk_size=1024,       # Characters per chunk
    chunk_overlap=200,     # Overlap between chunks
)

# 3. Retrieval with Filtering
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=corpus.name,
            # Filter to specific files:
            rag_file_ids=["file-id-1", "file-id-2"],
        )
    ],
    text="query",
    similarity_top_k=5,
)

# 4. Multiple Corpora
# Query across multiple knowledge bases:
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_resources=[
                rag.RagResource(rag_corpus="corpus-1"),
                rag.RagResource(rag_corpus="corpus-2"),
            ],
            similarity_top_k=5,
        ),
    )
)

# 5. Data Sources
# Import from Google Drive:
response = rag.import_files(
    corpus_name=corpus.name,
    paths=["https://drive.google.com/file/d/{file_id}"],
)

# Import from Slack:
# rag.import_files_async(..., source=rag.SlackChannelsSource(...))

# Import from Jira:
# rag.import_files_async(..., source=rag.JiraSource(...))
'''

print("📋 Advanced RAG Configuration Patterns:")
print(ADVANCED_PATTERNS)

## 8. Cleanup

In [ ]:
# @title Cleanup resources
CLEANUP = False  # @param {type:"boolean"}

if CLEANUP:
    # Delete RAG corpus (this also deletes all files in it)
    try:
        rag.delete_corpus(name=rag_corpus.name)
        print(f"✓ Deleted RAG corpus: {rag_corpus.display_name}")
    except Exception as e:
        print(f"Could not delete corpus: {e}")
    
    # Delete Cloud Storage bucket
    try:
        bucket = storage_client.bucket(BUCKET_NAME)
        bucket.delete(force=True)
        print(f"✓ Deleted bucket: {BUCKET_NAME}")
    except Exception as e:
        print(f"Could not delete bucket: {e}")
else:
    print("Skipping cleanup. Set CLEANUP=True to delete resources.")
    print(f"\n⚠️ Remember to delete these resources when done:")
    print(f"  - RAG Corpus: {rag_corpus.name}")
    print(f"  - GCS Bucket: {BUCKET_NAME}")

## Summary

In this notebook, you learned how to:

1. ✅ **Create a RAG corpus** with custom embedding configuration
2. ✅ **Upload and index documents** from Cloud Storage
3. ✅ **Perform direct retrieval** to find relevant contexts
4. ✅ **Generate RAG-grounded responses** using Gemini with retrieval tools
5. ✅ **Compare DIY vs managed RAG** approaches

### Key Takeaways

- **RAG Engine** provides a fully managed RAG solution
- **Automatic processing** handles chunking, embeddings, and indexing
- **Tool.from_retrieval()** seamlessly integrates RAG with Gemini
- **Multiple data sources** are supported (GCS, Drive, Slack, Jira)
- **Choose your approach** based on customization vs simplicity needs

---

## Chapter 2 Complete!

Congratulations! You've completed all the hands-on exercises for Chapter 2: Data Readiness and Accessibility.

### What You've Learned

| Notebook | Key Skills |
|----------|------------|
| 01 - Data Exploration | BigQuery profiling, data quality assessment |
| 02 - Document Processing | Gemini multimodal, Document AI patterns |
| 03 - Embeddings & Vector Search | Text embeddings, semantic similarity |
| 04 - RAG Context Assembly | Complete RAG pipeline, context assembly |
| 05 - RAG Engine | Managed RAG, production deployment |

### Next Steps

Continue to **Chapter 3** to learn about building AI Agents with ADK (Agent Development Kit)!